In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
import os.path
from funcs import read_config, get_groups, pull_clean, get_kpi2_respuestas, get_kpi3_retencion, get_kpi4_influencia

In [2]:
business_yelpp = pd.read_parquet('../../datasets/processed/bd/6_business_yelp.parquet.gz')
business_yelpp

,business_id,name,latitude,longitude,avg_stars,state_id
0,eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,27.955269,-82.456320,4.09,1
1,0bPLkL0QhhPO5kt1_EXmNQ,Zios Italian Market,27.916116,-82.760461,4.45,1
2,uI9XODGY_2_ieTE6xJ0myw,Roman Forum,28.046203,-82.505053,4.00,1
3,JgpnXv_0XhV3SfbfB50nxw,Joes Pizza,27.960514,-82.506127,4.17,1
4,pJfh3Ct8iL58NZa8ta-a5w,Top Shelf Sports Lounge,28.196252,-82.380615,4.51,1
...,...,...,...,...,...,...
12912,Df-DdXqssj7ZeD8ypAF7Og,Crux Cafe,39.894469,-75.071001,4.00,4
12913,qjtELTt9fdIwoi_xGNN21g,Chipotle Mexican Grill,39.943481,-74.964743,2.43,4
12914,BzPTw9daJW8ToqTc7QiHFw,Taco Bell,40.210771,-74.756588,2.67,4
12915,UBQAksw81m0sMrAd8g-ECg,El Zarape Restaurant,39.718717,-74.971143,4.00,4


In [3]:
#Data Pull and Functions
data_frames = pull_clean() 
state = data_frames.get('1_states.parquet')
categories = data_frames.get('2_categories.parquet')
# user_yelp = data_frames.get('3_user_yelp.parquet')
user_google = data_frames.get('4_user_google.parquet')
business_google = data_frames.get('5_business_google.parquet')
# business_yelp = data_frames.get('6_business_yelp.parquet')
categories_google = data_frames.get('7_categories_google.parquet')
# categories_yelp = data_frames.get('8_categories_yelp.parquet')
reviews_google = data_frames.get('9_reviews_google.parquet')
# reviews_yelp = data_frames.get('10_reviews_yelp.parquet')

1_states.parquet: (4, 2)
2_categories.parquet: (1297, 2)
4_user_google.parquet: (1363232, 5)
5_business_google.parquet: (24801, 6)
6_business_yelp.parquet: (12917, 6)
7_categories_google.parquet: (70726, 2)
9_reviews_google.parquet: (2437164, 8)


In [4]:
groups = get_groups(business_google)

In [5]:
categories

,categories_id,name
0,0,restaurant
1,1,sandwich shop
2,2,caterer
3,3,fast food restaurant
4,4,takeout restaurant
...,...,...
1292,1292,botanical gardens
1293,1293,newspapers & magazines
1294,1294,parking
1295,1295,georgian


In [13]:
groups.head(1)

,gmap_id,name,latitude,longitude,avg_stars,state_id,group
0,0x88d9ba5d65937567:0xbc27649cf513cc89,Bachata Rosa,25.848173,-80.299773,3.6,1,general


In [7]:
business_google.head(1)

,gmap_id,name,latitude,longitude,avg_stars,state_id,group
0,0x88d9ba5d65937567:0xbc27649cf513cc89,Bachata Rosa,25.848173,-80.299773,3.6,1,general


In [8]:
# Fusionar los dataframes en 'gmap_id'
merged_df = pd.merge(reviews_google, groups, on='gmap_id', how='inner')

# Fusionar con el dataframe de estados para obtener el nombre del estado
merged_df = pd.merge(merged_df, state, on='state_id', how='inner')

# Fusionar con el dataframe de categorías para obtener el nombre de la categoría
categories_df = pd.merge(categories, categories_google, on='categories_id', how='inner')

# Fusionar con el dataframe de categorías para obtener el nombre de la categoría
merged_df = pd.merge(merged_df, categories_df, on='gmap_id', how='inner')



In [12]:
merged_df.drop_duplicates()


,review_id,user_id,gmap_id,stars,sentiment,date,resp_sentiment,resp_date,name_x,latitude,longitude,avg_stars,state_id,group,state,categories_id,name_y
0,0,1.0899115226265579e+20,0x80c2c778e3b73d33:0xbdc58662a4a97d49,5,2,2021-01-06 05:12:07,0.0,2021-01-06 05:12:07,San Soo Dang,34.058092,-118.29213,4.4,2,general,California,108,korean restaurant
2,1,1.1129032221979622e+20,0x80c2c778e3b73d33:0xbdc58662a4a97d49,5,2,2021-02-09 05:47:28,0.0,2021-02-09 05:47:28,San Soo Dang,34.058092,-118.29213,4.4,2,general,California,108,korean restaurant
4,2,1.1264035744961195e+20,0x80c2c778e3b73d33:0xbdc58662a4a97d49,4,0,2020-03-08 05:04:42,0.0,2020-03-08 05:04:42,San Soo Dang,34.058092,-118.29213,4.4,2,general,California,108,korean restaurant
6,3,1.1744034972382365e+20,0x80c2c778e3b73d33:0xbdc58662a4a97d49,5,2,2019-03-07 05:56:56,0.0,2019-03-07 05:56:56,San Soo Dang,34.058092,-118.29213,4.4,2,general,California,108,korean restaurant
8,4,1.0058077083612353e+20,0x80c2c778e3b73d33:0xbdc58662a4a97d49,5,2,2017-05-16 05:01:41,0.0,2017-05-16 05:01:41,San Soo Dang,34.058092,-118.29213,4.4,2,general,California,108,korean restaurant
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9224800,2452771,1.170133647031647e+20,0x880fce33edd16c7b:0x44e066027b82ad8d,2,0,2015-06-24 18:25:59,0.0,2015-06-24 18:25:59,Noodles and Company,42.012992,-87.71181,4.2,3,general,Illinois,4,takeout restaurant
9224801,2452771,1.170133647031647e+20,0x880fce33edd16c7b:0x44e066027b82ad8d,2,0,2015-06-24 18:25:59,0.0,2015-06-24 18:25:59,Noodles and Company,42.012992,-87.71181,4.2,3,general,Illinois,66,salad shop
9224802,2452771,1.170133647031647e+20,0x880fce33edd16c7b:0x44e066027b82ad8d,2,0,2015-06-24 18:25:59,0.0,2015-06-24 18:25:59,Noodles and Company,42.012992,-87.71181,4.2,3,general,Illinois,93,pasta shop
9224803,2452771,1.170133647031647e+20,0x880fce33edd16c7b:0x44e066027b82ad8d,2,0,2015-06-24 18:25:59,0.0,2015-06-24 18:25:59,Noodles and Company,42.012992,-87.71181,4.2,3,general,Illinois,181,down home cooking restaurant
